In [9]:
# DAY 1 → Problem Definition & Simulation Strategy

In [18]:
#1. Objective

#Simualte external aerodynamics of a 25deg Ahmed body using steady RANS in Openfoam.
#Primary targets:
#- compute drag coefficents (Cd)
#-Study wake structure
#-Validate against literature Cd ~ 0.29 to 0.30


In [19]:
# 2. Geometry Definition

#Main dimensions:
#- Length (L) = 1.044m
#- Height (H) = 0288m
#- Width (W) = 0389m
#- Slant length (Ls) = 0.222m
#- Slant angle (theta) = 25deg


In [16]:
#3.Flow Conditions

#Assumptions:
#- Incompressible air
#- ρ = 1.225 kg/m³
#- μ = 1.789e−5 Pa·s
#- Freestream velocity U = 40 m/s
#Reynolds number estimate:

#        Re = (ρ U L) / μ

#Target Re ≈ O(10⁶), ensuring fully turbulent regime.


In [ ]:
## 4. Solver Selection

#Solver: simpleFoam

#Reason:
#- Steady-state RANS solver
#- Suitable for high Reynolds external flows
#- Computationally efficient compared to transient LES/DES

In [ ]:
## 5. Turbulence Model

#Model: kOmegaSST

#Reason:
#- Accurate near-wall prediction
#- Better separation handling than k-epsilon
#- Industry standard for automotive aerodynamics

In [ ]:
## 6. Modeling Strategy

#- Steady RANS approach
#- Moving ground boundary condition
#- Symmetry not used (full 3D domain)
#- Geometry created parametrically in Gmsh
#- Mesh will be generated in Gmsh (not snappyHexMesh)

In [ ]:
## 7. Expected Physical Behavior

#For 25° Ahmed body:
#- Separation occurs at rear slant
#- Moderate recirculation bubble
#- Expected Cd ≈ 0.29–0.30

In [22]:
import math

rho = 1.225
U = 40
L = 1.044
mu = 1.789e-5

Re = rho * U * L / mu
Re

2859474.5667970935

In [ ]:
# Day 2 – Geometry Construction & Debugging (Gmsh + OpenCASCADE)

In [ ]:
##1. Objective

#Create a clean, parametric 3D Ahmed body geometry (25deg slant) using gmsh with OpenCASCADE kernel.

In [ ]:
## 2. Geometry Strategy

#Approach:
#- Use OpenCASCADE kernel for solid modeling
#- Construct base rectangular block
#- Construct rear slanted wedge
#- Use Boolean operations to obtain single solid volume

# L = 1.044
#- H = 0.288
#- W = 0.389
#- Ls = 0.222
#- θ = 25°

#Slant height derived from:

#H_slant = Ls * tan(θ)

In [ ]:
## 3. Boolean Operation Strategy

#Initial attempt:
#- BooleanDifference used to carve slant from base block

#Issue observed:
#- Internal partitions created
#- Multiple volumes appeared
#- Tag conflicts generated OpenCASCADE errors

#Error example:
#- "OpenCASCADE point with tag already exists"
#- Curve loop duplication
#- Surface tag conflicts

#Root cause:
#- Reuse of low-number entity tags
#- Multiple geometry redefinitions in same script
#- Incorrect Boolean volume referencing

In [ ]:
## 4. Final Correct Approach

Solution implemented:
- Explicit parametric definition
- Controlled entity numbering
- Used BooleanUnion to fuse geometry
- Ensured single continuous volume
- Verified geometry via rotation and visibility check

Result:
- Clean slanted rear face
- Single solid volume
- No parser errors
- Geometry validated visually

In [ ]:
#Lesson learned

1. OpenCASCADE requires strict tag control.
2. BooleanDifference may create partitions if not handled carefully.
3. Parametric geometry improves reproducibility.
4. Always verify final volume count before meshing.

In [3]:
import math

Ls = 0.222
theta = math.radians(25)
Hslant = Ls * math.tan(theta)

Hslant

0.10352030011040969

## Final Geometry Visualization

![Ahmed Body Geometry](../figures/ahmed25_geometry.png)

In [ ]:
# Day 3 – Mesh Generation, OpenFOAM Conversion & Validation

In [ ]:
## 1. Objective

Generate a CFD-ready 3D mesh for the Ahmed body wind tunnel domain, 
convert it to OpenFOAM format, and validate mesh quality using checkMesh.

In [ ]:
## 2. Meshing Strategy

### Mesh Type
- Unstructured tetrahedral mesh
- 100% tetra elements

### Global Mesh Control
- Mesh.CharacteristicLengthMin = 0.05
- Mesh.CharacteristicLengthMax = 0.2

Purpose:
- Coarse mesh in far-field
- Controlled overall cell count

In [ ]:
## 3. Local Refinement Strategy

Distance-based refinement field applied on body surfaces:

- SizeMin = 0.02
- SizeMax = 0.2
- DistMin = 0.05
- DistMax = 0.5

Purpose:
- Fine mesh near Ahmed body
- Gradual transition outward
- Resolve boundary layer region and wake origin


In [ ]:
## 4. Mesh Statistics (Before Conversion)

- Points: 8,260
- Cells: 40,316
- Element Type: Tetrahedra only
- Single fluid volume

This mesh is suitable for:
- Workflow validation
- Initial steady RANS simulation
- Not yet suitable for high-fidelity drag prediction

In [ ]:
## 5. Physical Groups (Patch Mapping)

### Volume
- fluid

### Surfaces
- inlet
- outlet
- top
- bottom
- side1
- side2
- body

These physical groups are required for proper OpenFOAM patch generation.


In [ ]:
## 6. Mesh Export & Conversion

Mesh exported as:
- Gmsh MSH Version 2 ASCII

Conversion tool:
- gmshToFoam

Important workflow rule:
- gmshToFoam must be executed from case root directory.

In [ ]:
## 7. checkMesh Validation

Results summary:

- Regions: 1 (Correct)
- Max aspect ratio: 7.80 (OK)
- Max skewness: 0.89 (OK)
- Max non-orthogonality: 65.1° (Acceptable for RANS)
- No negative volumes
- No topology errors

Verdict:
Mesh OK – Solver Ready

In [ ]:
## 8. Lessons Learned

1. OpenFOAM utilities require correct directory structure.
2. MSH must be exported as Version 2 ASCII.
3. Physical groups in Gmsh directly define OpenFOAM patches.
4. Boolean operations must be validated before meshing.
5. checkMesh is mandatory before solving.
6. Clipping plane is useful for internal mesh inspection.

In [ ]:
## 9. Engineering Reflection

The mesh is intentionally moderate in size (~40k cells) 
to validate solver pipeline before increasing resolution.

Next step:
- Define boundary conditions
- Configure turbulence model
- Run simpleFoam

## Final Mesh Visualization

![Ahmed Body Geometry](../figures/ahmed25_mesh_1.png)

![Ahmed Body Geometry](../figures/ahmed25_mesh_2.png)

In [ ]:
#mesh quality metrics

Mesh stats
    points:           8260
    faces:            84072
    internal faces:   77192
    cells:            40316
    faces per cell:   4
    boundary patches: 7
    point zones:      0
    face zones:       0
    cell zones:       1